In [48]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,MaxPool2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.optimizers.legacy import RMSprop,Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical


import os
import random
import zipfile
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image
plt.style.use('ggplot')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,precision_score,recall_score

In [2]:
try:
    GPUlist = tf.config.list_physical_devices('GPU')
    print(GPUlist)
    for i in GPUlist:
        tf.config.experimental.set_memory_growth(i, True)
except:
    pass

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
trainCSV = pd.read_csv('train.csv')
testCSV = pd.read_csv('test.csv')
trainCSV.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
trainY = trainCSV['label']
trainX = trainCSV.drop(columns=['label'], axis=1)
trainX = trainX/255.0
testX = testCSV/255.0

In [38]:
trainX = trainX.values.reshape(-1,28,28,1)
testX = testX.values.reshape(-1,28,28,1)
trainY = to_categorical(trainY, num_classes = 10)
trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size = 0.2)

print("trainX:", trainX.shape)
print("testX:", testX.shape)
print("trainY:", trainY.shape)

trainX: (33600, 28, 28, 1)
testX: (28000, 28, 28, 1)
trainY: (33600, 10)


In [39]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size = (7,7), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size = (5,5), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size = (3,3), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size = (3,3), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 28, 28, 32)        1600      
                                                                 
 batch_normalization_10 (Bat  (None, 28, 28, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_12 (Conv2D)          (None, 28, 28, 32)        25632     
                                                                 
 batch_normalization_11 (Bat  (None, 28, 28, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                      

In [57]:
model.compile(optimizer=Adam(lr=0.01), loss="categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
# earlystop = EarlyStopping(monitor='val_loss',patience=4,verbose=1)
callbacks = [learning_rate_reduction] #+ [earlystop]

In [60]:
train_datagen = ImageDataGenerator(rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=False,
                                   fill_mode='nearest')

In [61]:
epochs = 30
batch_size = 64

history = model.fit(train_datagen.flow(trainX,trainY, batch_size=batch_size),
    epochs = epochs, validation_data = (valX , valY) ,
    steps_per_epoch=trainX.shape[0] // batch_size,callbacks=callbacks)


Epoch 1/30
525/525 [==============================] - 13s 24ms/step - loss: 0.0513 - accuracy: 0.9868 - val_loss: 0.0382 - val_accuracy: 0.9913 - lr: 7.8125e-05
Epoch 2/30
525/525 [==============================] - 13s 24ms/step - loss: 0.0505 - accuracy: 0.9869 - val_loss: 0.0387 - val_accuracy: 0.9917 - lr: 7.8125e-05
Epoch 3/30
525/525 [==============================] - 12s 24ms/step - loss: 0.0519 - accuracy: 0.9872 - val_loss: 0.0385 - val_accuracy: 0.9915 - lr: 7.8125e-05
Epoch 4/30
525/525 [==============================] - 13s 25ms/step - loss: 0.0514 - accuracy: 0.9874 - val_loss: 0.0375 - val_accuracy: 0.9915 - lr: 7.8125e-05
Epoch 5/30
523/525 [============================>.] - ETA: 0s - loss: 0.0527 - accuracy: 0.9869
Epoch 5: ReduceLROnPlateau reducing learning rate to 3.9062499126885086e-05.
525/525 [==============================] - 13s 24ms/step - loss: 0.0526 - accuracy: 0.9870 - val_loss: 0.0373 - val_accuracy: 0.9914 - lr: 7.8125e-05
Epoch 6/30
525/525 [=============

In [63]:
results = model.predict(testX)

875/875 [==============================] - 6s 7ms/step


In [64]:
len(results)

28000

In [65]:
results = np.argmax(results,axis = 1)
results = pd.DataFrame({"ImageId":[i for i in range(1,28001)], "Label":results})
results.to_csv("results.csv",index=False)